# Predicting Judicial Decisions of the European Court of Human Rights

In [1]:
import numpy as np
import re
import os
import copy

In [2]:
import sklearn.metrics as sm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [3]:
import spacy
from spacy.lang import en

In [4]:
scoring = {'accuracy': make_scorer(sm.accuracy_score),
           'precision': make_scorer(sm.precision_score),
           'recall': make_scorer(sm.recall_score),
           'f1': make_scorer(sm.f1_score)}

In [5]:
def read_dataset(PATH):
    X_dataset = {}
    Y_dataset = {}
    for path, dirs, files in os.walk(PATH):
        for filename in files:
            fullpath = os.path.join(path, filename)
            if "both" not in fullpath:
                with open(fullpath, 'r', encoding="utf8") as file:
                    X_dataset, Y_dataset = add_file_to_dataset(fullpath, X_dataset, Y_dataset, file.read())

    return X_dataset, Y_dataset       

In [6]:
def add_file_to_dataset(fullpath, x_dataset, y_dataset, file):
    article = extract_article(fullpath)
    file = preprocess(file)
    if article not in x_dataset.keys() :
        x_dataset[article] = []
        y_dataset[article] = []
    x_dataset[article] = x_dataset[article] + [file]
    label = 0 if "non-violation" in fullpath else 1
    y_dataset[article] = y_dataset[article] + [label]
    return x_dataset, y_dataset  

In [7]:
def extract_article(path): 
    pattern = r"(Article\d+)"
    result = re.search(pattern, path)
    article = result.group(1)
    return article

### Preprocessing 

In [8]:
def preprocess(file): 
    file = extract_paragraphs(file)
    return file

In [9]:
def extract_paragraphs(file): 
    file = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', file)
    pat = r'(PROCEDURE\s*\n.+?)?((THE CIRCUMSTANCES OF THE CASE\s*\n.+?RELEVANT DOMESTIC LAW.+?)|(\n(AS TO THE FACTS|THE FACTS|FACTS)\s*\n.+?))(\nIII\.|THE LAW\s*\n|PROCEEDINGS BEFORE THE COMMISSION\s*\n|ALLEGED VIOLATION OF ARTICLE [0-9]+ OF THE CONVENTION \s*\n)'
    result = re.search(pat, file, re.S |  re.IGNORECASE)
    if result is None:
        print(repr(file))
    content = ""
    if result.group(1) is not None:
        content += result.group(1)
    content += result.group(2)
    return content

### Loading the data

In [10]:
base_path = "Datasets\\Human rights dataset"

In [11]:
X_train_docs, Y_train_docs = read_dataset(base_path + "/train")
X_test_docs, Y_test_docs = read_dataset(base_path + "/test20")

In [12]:
X_train_docs.keys()

dict_keys(['Article10', 'Article11', 'Article12', 'Article13', 'Article14', 'Article18', 'Article2', 'Article3', 'Article4', 'Article5', 'Article6', 'Article7', 'Article8'])

In [13]:
X_test_docs.keys()

dict_keys(['Article10', 'Article11', 'Article12', 'Article13', 'Article14', 'Article18', 'Article2', 'Article3', 'Article4', 'Article5', 'Article6', 'Article7', 'Article8'])

### Combining all the articles according to class

In [14]:
X_train = X_train_docs["Article2"] + X_train_docs["Article3"] + X_train_docs["Article5"] + X_train_docs["Article6"] + X_train_docs["Article8"] + X_train_docs["Article10"] + X_train_docs["Article11"] + X_train_docs["Article13"] + X_train_docs["Article14"]

In [15]:
X_test = X_test_docs["Article2"] + X_test_docs["Article3"] + X_test_docs["Article5"] + X_test_docs["Article6"] + X_test_docs["Article8"] + X_test_docs["Article10"] + X_test_docs["Article11"] + X_test_docs["Article13"] + X_test_docs["Article14"]

In [16]:
Y_train = Y_train_docs["Article2"] + Y_train_docs["Article3"] + Y_train_docs["Article5"] + Y_train_docs["Article6"] + Y_train_docs["Article8"] + Y_train_docs["Article10"] + Y_train_docs["Article11"] + Y_train_docs["Article13"] + Y_train_docs["Article14"]

In [17]:
Y_test = Y_test_docs["Article2"] + Y_test_docs["Article3"] + Y_test_docs["Article5"] + Y_test_docs["Article6"] + Y_test_docs["Article8"] + Y_test_docs["Article10"] + Y_test_docs["Article11"] + Y_test_docs["Article13"] + Y_test_docs["Article14"]

In [18]:
len(X_train), len(Y_train), len(X_test), len(Y_test)

(3131, 3131, 784, 784)

### Tokenization with TfIdfVectorizer and Classification with Logistic Regression

In [19]:
# tokenize the doc and lemmatize its tokens
lemmatizer = spacy.lang.en.English()
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])

#### Test20 evaluation

In [20]:
vect = TfidfVectorizer(ngram_range=(2,4), lowercase=True, tokenizer=my_tokenizer, max_features=600000, min_df=3)
term_doc_matrix = vect.fit_transform(X_train).toarray()    
test_term_doc = vect.transform(X_test).toarray()

In [21]:
# from sklearn.utils import shuffle
# X_train, term_doc_matrix, Y_train = shuffle(X_train, term_doc_matrix, Y_train)

In [22]:
X_train_new = SelectKBest(chi2, k=300000).fit_transform(term_doc_matrix, Y_train)
X_train_new.shape

(3131, 300000)

In [23]:
classifier = LogisticRegression(solver='lbfgs')        
classifier.fit(term_doc_matrix, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
y_test_pred = classifier.predict(test_term_doc)

In [25]:
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(Y_test, y_test_pred), 2))
print("Accuracy score =", sm.accuracy_score(Y_test, y_test_pred))
print("Recall score =", sm.recall_score(Y_test, y_test_pred))
print("Precision score =", sm.precision_score(Y_test, y_test_pred))
print("F1 score =", sm.f1_score(Y_test, y_test_pred))

Logistic regression performance:
Mean absolute error = 0.21
Accuracy score = 0.7857142857142857
Recall score = 0.7627551020408163
Precision score = 0.7994652406417112
F1 score = 0.7806788511749346


In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_test_pred)

array([[317,  75],
       [ 93, 299]], dtype=int64)

In [27]:
tn, fp, fn, tp = confusion_matrix(Y_test, y_test_pred).ravel()
(tn, fp, fn, tp)

(317, 75, 93, 299)

0.785 accuracy,
0.762 recall,
0.799 precision,
0.780 f1

### Tokenization with TfIdfVectorizer and Classification with Linear SVC

In [19]:
vect = TfidfVectorizer(ngram_range=(2,4), lowercase=True, max_features=600000, min_df=3)
term_doc_matrix = vect.fit_transform(X_train).toarray()  
test_term_doc = vect.transform(X_test).toarray()

In [20]:
# from sklearn.utils import shuffle
# X_train, term_doc_matrix, Y_train = shuffle(X_train, term_doc_matrix, Y_train)

In [21]:
X_train_new = SelectKBest(chi2, k=300000).fit_transform(term_doc_matrix, Y_train)
X_train_new.shape

(3131, 300000)

In [22]:
classifier = svm.LinearSVC(C=0.1, max_iter=1500)             
classifier.fit(term_doc_matrix, Y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1500,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [23]:
y_test_pred = classifier.predict(test_term_doc)

In [24]:
# Compute performance metrics
print("Logistic regression performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(Y_test, y_test_pred), 2))
print("Accuracy score =", sm.accuracy_score(Y_test, y_test_pred))
print("Accuracy score =", sm.recall_score(Y_test, y_test_pred))
print("Accuracy score =", sm.precision_score(Y_test, y_test_pred))
print("F1 score =", sm.f1_score(Y_test, y_test_pred))

Logistic regression performance:
Mean absolute error = 0.22
Accuracy score = 0.7806122448979592
Accuracy score = 0.7525510204081632
Accuracy score = 0.7972972972972973
F1 score = 0.7742782152230971


0.780 accuracy,
0.752 recall,
0.797 precision,
0.774 f1

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_test_pred)

array([[317,  75],
       [ 97, 295]], dtype=int64)

In [26]:
tn, fp, fn, tp = confusion_matrix(Y_test, y_test_pred).ravel()
(tn, fp, fn, tp)

(317, 75, 97, 295)